In [33]:
from __future__ import print_function, division
import os 
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import torch
import torchvision
from torchvision import transforms, datasets, models
import matplotlib.pyplot as plt
import time
import copy
cudnn.benchmark=True
plt.ion()

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [34]:
im_mean=np.array([0.485, 0.456, 0.406])
im_std=np.array([0.229, 0.224, 0.225])
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(im_mean, im_std)
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(im_mean, im_std)
    ]),
}

data_dir = '/Users/rafikhaliqi/handson-ml2/handson-ml2/MAU-AML-labs/1-computer-vision-lab/hymenoptera_data'
image_dataset = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloader = {x:torch.utils.data.DataLoader(image_dataset[x], batch_size=4, shuffle=True, num_workers=4)
                for x in ['train', 'val']}

dataset_size={x: len(image_dataset[x]) for x in ['train', 'val']}
class_names=image_dataset['train'].classes
print('classes:', class_names, '\nsize', dataset_size)

classes: ['ants', 'bees'] 
size {'train': 244, 'val': 153}


In [35]:
def imshow(inp, title=None):
    inp=inp.numpy().transpose((1,2,0))
    im_mean=np.array([0.485, 0.456, 0.406])
    im_std=np.array([0.229, 0.224, 0.225])
    inp=im_std*inp+im_mean
    inp=np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)
#inputs, classes=next(iter(dataloader['train']))
#out=torchvision.utils.make_grid(inputs)
#imshow(out, title=[class_names[x] for x in classes])

In [36]:
def train_model(model, criterion, optimizer):
    since = time.time()
    best_acc = 0.0
    model.train()   # Set model to train mode
    running_loss = 0.0
    running_corrects = 0
    
    for inputs, labels in dataloader['train']:
        inputs = inputs.to(device)
        labels = labels.to(device)
    
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        
        # zero the parameter gradients
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        
    epoch_loss = running_loss / dataset_size['train']
    epoch_acc = running_corrects.double() / dataset_size['train']

    print('train Loss: {:.4f} Acc: {:.4f}'.format(
         epoch_loss, epoch_acc))

In [37]:
def test_model(model, criterion):
    since = time.time()
    best_acc = 0.0
    model.eval()   # Set model to evaluate mode
    running_loss = 0.0
    running_corrects = 0
    
    for inputs, labels in dataloader['val']:
        inputs = inputs.to(device)
        labels = labels.to(device)
    
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        
    epoch_loss = running_loss / dataset_size['val']
    epoch_acc = running_corrects.double() / dataset_size['val']

    print('test Loss: {:.4f} Acc: {:.4f}'.format(
        epoch_loss, epoch_acc))

In [38]:
def visualize_model(model, num_images=3):
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloader['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    return

In [39]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features # Get what the final layer should receive

# Here the size of each output sample is set to 2 (the class names)
# Create a new final layer
model_ft.fc = nn.Linear(num_ftrs, len(class_names))

model_ft = model_ft.to(device)

In [40]:
num_epochs = 3

criterion = nn.CrossEntropyLoss()

# We are optimizing
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    train_model(model_ft, criterion, optimizer_ft)
    test_model(model_ft, criterion)
    scheduler.step()

Epoch 0/2


KeyboardInterrupt: 

In [ ]:
model_ft=torchvision.models.resnet18(pretrained=True)
for param in model_ft.parameters():
    param.requires_grad=False

num_ftrs=model_ft.fc.in_features
model_ft.fc=nn.Linear(num_ftrs, 2)
model_ft=model_ft.to(device)
criterion=nn.CrossEntropyLoss()
optimizer_ft=optim.SGD(model_ft.fc.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler=lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
model_ft=train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=3)

In [ ]:
model_ft

In [ ]:
def show_metrics(metrics):
    plt.figure(figsize=(12, 8))
    plt.plot(metrics['train'], label='train')
    plt.plot(metrics['val'], label='val')
    plt.grid()
    plt.legend()
    plt.show()